# Cavity Circuit Representations for the two micromachined cavity coupler

In [1]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

## Eigenvector / eigenvalue calculations

In [2]:
def two_cavity_one_coupler(L, C, Lc, Cc, LJ, debug=False):
    """
    Compute the eigenvalues and eigenvectors for the three stage
    circuit describing the two lowest modes of the cavity + JJ
    """
    # Inductance matrix
    LJtot = 1. / LJ + 1. / Lc
    L11 = 1. / L[0] + LJtot
    L22 = 1. / L[1] + LJtot
    Lmat = np.array([[L11, -LJtot], 
                     [-LJtot, L22]])
    
    # Capacitance matrix
    Cmat = np.array([[C[0] + Cc,        -Cc],
                    [-Cc,         C[1] + Cc]])
    
    # Solve the generalized eigenvalue problem
    # A * x = l * B * x
    # L^-1 * x - w^2 * C * x
    evals, evecs = scipy.linalg.eig(Lmat, b=Cmat)
    
    # Print the L^-1 and C matrices
    if debug:
        print(f'L^-1:\n{Lmat}')
        print(f'C:\n{Cmat}')
    
    return np.real(np.sqrt(evals)), evecs

In [3]:
def two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ, debug=False,
                                 coupling : str = 'both'):
    """
    Compute the eigenvalues and eigenvectors for the three stage
    circuit describing the two lowest modes of the cavity + JJ
    """
    if coupling == 'both':
        Lsterm = 1. / Ls
        Csterm = Cs
    elif coupling == 'C':
        Lsterm = 0
        Csterm = Cs
    elif coupling == 'L':
        Lsterm = 1. / Ls
        Csterm = 0
    else:
        raise ValueError(f'coupling ({coupling}) not recognized.')
        
    # Inductance matrix
    LJtot = 1. / LJ + Lsterm
    L11 = 1. / L[0] + LJtot
    L22 = 1. / L[1] + LJtot
    L44 = 1. / L[1] + Lsterm
    Lmat = np.array([[L11,    -LJtot,     0,        0],
                     [-LJtot,  L22,   -LJtot,       0],
                     [0,     -LJtot,    L22, -Lsterm],
                     [0,        0,     -Lsterm,    L44]])
    
    # Capacitance matrix
    C11 = C[0] + Csterm
    C22 = Csterm + CJ
    C44 = C[1] + Csterm
    Ctot = Csterm + CJ
    Cmat = np.array([[C11,  -Ctot,    0,         0],
                    [-Ctot,  C22,  -Ctot,        0],
                    [0,     -Ctot,   C22,  -Csterm],
                    [0,        0,   -Csterm,   C44]])
    
    # Solve the generalized eigenvalue problem
    # A * x = l * B * x
    # L^-1 * x - w^2 * C * x
    evals, evecs = scipy.linalg.eig(Lmat, b=Cmat)
    
    # Print the L^-1 and C matrices
    if debug:
        print(f'L^-1:\n{Lmat}')
        print(f'C:\n{Cmat}')
    
    return np.real(np.sqrt(evals)), evecs

In [4]:
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cs = 50e-15
CJ = 2e-15
LJ = 1e-9
Ls = 4e-9
ws, evs = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ)
# ws, evs = two_cavity_one_coupler(L, C, Lc, Cc, LJ)

In [5]:
ws / (2*np.pi*1e9)

array([25.39431618, 18.81225134, 18.81225134,  5.93569578])

## Plotting macros

In [6]:
def set_axes_fonts(ax, fsize):
    """
    Set axes font sizes because it should be abstracted away
    """
    for tick in ax.get_xticklabels():
        tick.set_fontsize(fsize)
    for tick in ax.get_yticklabels():
        tick.set_fontsize(fsize)

In [7]:
def set_xaxis_rot(ax, angle=45):
    """
    Rotate the x-axis labels
    """
        
    for tick in ax.get_xticklabels():
        tick.set_rotation(angle)

In [8]:
def set_leg_outside(ax, fsize):
    """
    Sets the legend location outside
    """
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    # Put a legend to the right of the current axis
    hdls, legs = ax.get_legend_handles_labels()
    leg = ax.legend(hdls, legs, loc='center left', bbox_to_anchor=(1, 0.5), \
                    fontsize=fsize, framealpha=0.)

    return leg

## Inspect the eigenvalues and eigenvectors vs. $L_J$

### Eigenvectors vs. $\phi$

In [19]:
# Start the figure
fig, ax = plt.subplots(1, 2, figsize=(16, 4), tight_layout=True)
fsize = 20; lsize = 16
dstr = datetime.datetime.today().strftime('%y%m%d')

L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cc = 50e-15
Lc = 4e-9
# ws, evs = two_cavity_one_coupler(L, C, Lc, Cc, LJ)
LJs = np.logspace(-2, 2, 5) * 1e-9

# Compute the eigenvalues and eigenvectors on-the-fly
for idx, LJ in enumerate(LJs):
    evals, evecs = two_cavity_one_coupler(L, C, Lc, Cc, LJ)
    for cidx in range(2):
        ax[cidx].plot([1, 2],
                     evecs[:, cidx], label=r'$L_J$ = %.2f nH' % (LJ / 1e-9))

        ax[cidx].set_xlabel(r'$\phi$', fontsize=fsize)
        ax[cidx].set_ylabel(r'$\xi_{%d}$' % (cidx + 1), fontsize=fsize)
        set_axes_fonts(ax[cidx], fsize)
        ax[cidx].set_ylim([-1, 1])
        
set_leg_outside(ax[1], lsize)
fig.savefig(f'../figs/evecs_three_stage_{dstr}.pdf',
           format='pdf')

### Eigenvalues vs. $L_J$

In [10]:
# Compute the eigenvalues
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cc = 500e-15
LJ = 1e-9
Lc = 4e-9
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
evs = np.zeros([2, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler(L, C, Lc, Cc, LJ)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
fig_ev, ax_ev = plt.subplots(1, 1, tight_layout=True)
for widx, w in enumerate(evs):
    label = f'Mode-{widx+1}'
    ax_ev.plot(LJsfull / 1e-9, w / (2*np.pi*1e9), 'o-', label=label)
ax_ev.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax_ev.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax_ev.set_xscale('log')
#ax_ev.set_yscale('log')
# ax_ev.set_ylim([0.3 * fa, 1.1 * fb])
# ax_ev.axhline(y=0, ls='--', color='k')
# ax_ev.axvline(x=1, ls=':', color='k')
# ax_ev.axhline(y=fa, ls='--', color='r')
set_axes_fonts(ax_ev, fsize)
set_leg_outside(ax_ev, lsize)
fig_ev.savefig(f'../figs/evals_three_stage_{dstr}.pdf',
              format='pdf')

## Plot the series model results

### Capacitive and inductive coupling + sliders

In [11]:
%matplotlib notebook
# run again
%matplotlib notebook
%matplotlib inline
fsize = 20; lsize = 16
from importlib import reload
import matplotlib.pyplot as plt
reload(plt)
from matplotlib.widgets import Button, Slider

fig, ax = plt.subplots(1, 1) #, tight_layout=True)
fig.subplots_adjust(bottom=0.35)

# Compute the eigenvalues
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cs = 700e-15
LJ = 1e-9
Ls = 5e-9
CJ = 2e-15
coupling = 'both'
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
evs0 = np.zeros([len(LJsfull), 4])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    evs0[Lidx, :] = np.sort(evv)
    evs = np.copy(evs0)
lines = ax.plot(LJsfull / 1e-9, evs / (2*np.pi*1e9), 'o-')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax.set_xscale('log')
set_axes_fonts(ax, fsize)

# Position the sliders such that they are stacked vertically
ax_L = fig.add_axes([0.25, 0.25, 0.65, 0.03])
ax_C = fig.add_axes([0.25, 0.2, 0.65, 0.03])
ax_Ls = fig.add_axes([0.25, 0.15, 0.65, 0.03])
ax_Cs = fig.add_axes([0.25, 0.1, 0.65, 0.03])
ax_CJ = fig.add_axes([0.25, 0.05, 0.65, 0.03])

# create the sliders
Lslider = Slider(
    ax_L, r'$L_{1,2}$', 0.01e-9, 50e-9,
    valinit=10e-9, valstep=0.01e-9,
    color="blue"
)
Cslider = Slider(
    ax_C, r'$C_{1,2}$', 2e-15, 2000e-15,
    valinit=200e-15, valstep=1e-15,
    color="blue"
)
Lsslider = Slider(
    ax_Ls, r'$L_s$', 0.01e-9, 50e-9,
    valinit=5e-9, valstep=0.01e-9,
    color="blue"
)
Csslider = Slider(
    ax_Cs, r'$C_s$', 2e-15, 2000e-15,
    valinit=700e-15, valstep=1e-15,
    color="blue"
)
CJslider = Slider(
    ax_CJ, r'$C_J$', 1e-15, 20e-15,
    valinit=2e-15, valstep=1e-15,
    color="blue"
)

def update(val):
    for Lidx, LJ in enumerate(LJsfull):
        evv, _ = two_cavity_one_coupler_series(Lslider.val, Cslider.val,
                                               Lsslider.val, Csslider.val,
                                               LJ, CJslider.val,
                                               coupling=coupling)
        evs[Lidx, :] = np.sort(evv)
    for idx, line in enumerate(lines):
        line.set_ydata(evv[:, idx])
    fig.canvas.draw_idle()

Lslider.on_changed(update)
Cslider.on_changed(update)
Lsslider.on_changed(update)
Csslider.on_changed(update)
CJslider.on_changed(update)

ax_reset = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(ax_reset, 'Reset', hovercolor='0.975')

def reset(event):
    Lslider.reset()
    Cslider.reset()
    Lsslider.reset()
    Csslider.reset()
    CJslider.reset()
    l.set_ydata(evs0)
button.on_clicked(reset)

0

In [12]:
lines

### Capacitive and inductive coupling

In [13]:
# Compute the eigenvalues
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cs = 700e-15
LJ = 1e-9
Ls = 5e-9
CJ = 2e-15
coupling = 'both'
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
evs = np.zeros([4, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
fig_ev, ax_ev = plt.subplots(1, 1, tight_layout=True)
for widx, w in enumerate(evs):
    label = f'Mode-{widx+1}'
    ax_ev.plot(LJsfull / 1e-9, w / (2*np.pi*1e9), 'o-', label=label)
ax_ev.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax_ev.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax_ev.set_xscale('log')
#ax_ev.set_yscale('log')
# ax_ev.set_ylim([0.3 * fa, 1.1 * fb])
# ax_ev.axhline(y=0, ls='--', color='k')
# ax_ev.axvline(x=1, ls=':', color='k')
# ax_ev.axhline(y=fa, ls='--', color='r')
set_axes_fonts(ax_ev, fsize)
set_leg_outside(ax_ev, lsize)
fig_ev.savefig(f'../figs/evals_three_stage_{dstr}.pdf',
              format='pdf')

### Capactive coupling

In [14]:
# Compute the eigenvalues
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cs = 700e-15
LJ = 1e-9
Ls = 5e-9
CJ = 2e-15
coupling = 'C'
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
evs = np.zeros([4, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
fig_ev, ax_ev = plt.subplots(1, 1, tight_layout=True)
for widx, w in enumerate(evs):
    label = f'Mode-{widx+1}'
    ax_ev.plot(LJsfull / 1e-9, w / (2*np.pi*1e9), 'o-', label=label)
ax_ev.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax_ev.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax_ev.set_xscale('log')
#ax_ev.set_yscale('log')
# ax_ev.set_ylim([0.3 * fa, 1.1 * fb])
# ax_ev.axhline(y=0, ls='--', color='k')
# ax_ev.axvline(x=1, ls=':', color='k')
# ax_ev.axhline(y=fa, ls='--', color='r')
set_axes_fonts(ax_ev, fsize)
set_leg_outside(ax_ev, lsize)
fig_ev.savefig(f'../figs/evals_three_stage_{dstr}.pdf',
              format='pdf')

### Inductive coupling

In [15]:
# Compute the eigenvalues
L = np.array([10e-9, 10e-9])
C = np.array([200e-15, 200e-15])
Cs = 70e-15
Ls = 3e-9
CJ = 5e-15
coupling = 'L'
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
# LJsfull = np.linspace(1, 20, 50) * 1e-9
evs = np.zeros([4, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
fig_ev, ax_ev = plt.subplots(1, 1, tight_layout=True)
for widx, w in enumerate(evs):
    label = f'Mode-{widx+1}'
    ax_ev.plot(LJsfull / 1e-9, w / (2*np.pi*1e9), 'o-', label=label)
ax_ev.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax_ev.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax_ev.set_xscale('log')
#ax_ev.set_yscale('log')
ax_ev.set_ylim([-1, 20])
# ax_ev.axhline(y=0, ls='--', color='k')
# ax_ev.axvline(x=1, ls=':', color='k')
# ax_ev.axhline(y=fa, ls='--', color='r')
set_axes_fonts(ax_ev, fsize)
set_leg_outside(ax_ev, lsize)
fig_ev.savefig(f'../figs/evals_three_stage_{dstr}.pdf',
              format='pdf')

### Eigenvectors of series inductively coupled

In [18]:
# Start the figure
fig, ax = plt.subplots(2, 4, figsize=(36, 10), tight_layout=True)
fsize = 20; lsize = 16
dstr = datetime.datetime.today().strftime('%y%m%d')

L = np.array([10e-9, 10e-9])
C = np.array([30e-15, 30e-15])
wab = 1. / np.sqrt(L * C)
print(f'wa, wb: {wab / (1e9 * 2*np.pi)}')
Cc = 50e-15
Lc = 4e-9
# ws, evs = two_cavity_one_coupler(L, C, Lc, Cc, LJ)
LJs = np.logspace(-1, 1, 3) * 1e-9
wc = 1. / np.sqrt(LJs * Cc)
print(f'wc: {wc / (1e9 * 2*np.pi)}')

# Compute the eigenvalues and eigenvectors on-the-fly
for Lidx, LJ in enumerate(LJs):
    _, evecs = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    for cidx in range(4):
        ax[0, cidx].plot([1, 2, 3, 4],
                      np.real(evecs[:, cidx]), label=r'$L_J$ = %.2f nH' % (LJ / 1e-9))

        ax[0, cidx].set_xlabel(r'$\phi$', fontsize=fsize)
        ax[0, cidx].set_ylabel(r'$\Re\{\xi_{%d}\}$' % (cidx + 1), fontsize=fsize)
        set_axes_fonts(ax[0, cidx], fsize)
        ax[0, cidx].set_ylim([-1, 1])
        
#     for cidx in range(4):
#         ax[1, cidx].plot([1, 2, 3, 4],
#                       np.imag(evecs[:, cidx]), label=r'$L_J$ = %.2f nH' % (LJ / 1e-9))
# 
#         ax[1, cidx].set_xlabel('$\phi$', fontsize=fsize)
#         ax[1, cidx].set_ylabel(r'$\Im\{\xi_{%d}\}$' % (cidx + 1), fontsize=fsize)
#         set_axes_fonts(ax[1, cidx], fsize)
#         ax[1, cidx].set_ylim([-1, 1])

set_leg_outside(ax[0, 3], lsize)

# Plot the eigenvalues
coupling = 'L'
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
# LJsfull = np.linspace(1, 20, 50) * 1e-9
evs = np.zeros([4, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler_series(L, C, Ls, Cs, LJ, CJ,
                                           coupling=coupling)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
for widx, w in enumerate(evs):
    label = f'Mode-{widx+1}'
    ax[1, 0].plot(LJsfull / 1e-9, w / (2*np.pi*1e9), 'o-', label=label)
ax[1, 0].set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax[1, 0].set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
ax[1, 0].set_xscale('log')
#ax_ev.set_yscale('log')
ax[1, 0].set_ylim([-1, 20])
# ax_ev.axhline(y=0, ls='--', color='k')
# ax_ev.axvline(x=1, ls=':', color='k')
# ax_ev.axhline(y=fa, ls='--', color='r')
set_axes_fonts(ax[1, 0], fsize)
set_leg_outside(ax[1, 3], lsize)

wa, wb: [9.18881492 9.18881492]
wc: [71.17625434 22.5079079   7.11762543]


In [17]:
# Compute the eigenvalues
LJsfull = np.logspace(np.log10(0.1), np.log10(100), 25) * 1e-9
evs = np.zeros([2, len(LJsfull)])
for Lidx, LJ in enumerate(LJsfull):
    evv, _ = two_cavity_one_coupler(L, C, Lc, Cc, LJ)
    evs[:, Lidx] = np.sort(evv)
    
# Plot the eigenvalues
fig_ev, ax_ev = plt.subplots(1, 1, tight_layout=True)
ax_ev.plot(LJsfull / 1e-9, (evs[1] - evs[0]) / (1e6 * 2*np.pi), 'o-')
ax_ev.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax_ev.set_ylabel(r'Mode Splitting (1-2) [MHz]', fontsize=fsize)
ax_ev.set_xscale('log')
#ax_ev.set_yscale('log')
# ax_ev.set_ylim([0, 10])
set_axes_fonts(ax_ev, fsize)
fig_ev.savefig(f'../figs/evals_three_stage_{dstr}.pdf',
              format='pdf')